In [1]:
import os
from itertools import permutations 
import matplotlib.pyplot as plot

stats = ("length", "visitedStates", "processedStates", "depth", "time")
strategies = ("bfs", "dfs", "astr")
lrudVariants = ("RDUL", "RDLU", "DRUL", "DRLU", "LUDR", "LURD", "ULDR", "ULRD")
#lrudVariants = ("".join(perm) for perm in permutations("LRUD"))
astrVariants = ("hamm", "manh")

def packFilename(filename, strategy, variant, type):
	name, extension = os.path.splitext(filename)
	return "{}_{}_{}_{}{}".format(name, strategy, variant, type, extension)

def unpackFilename(filename):
	name, extension = os.path.splitext(filename)
	puzzleSize, iterations, puzzleId, strategy, variant, type = name.split("_") 
	return iterations, strategy, variant, type

def mean(x):
	length = len(x)
	if length != 0:
		return sum(x) / length
	else:
		return 0

def solvePuzzles(puzzlesDirectory, solutionsDirectory, statsDirectory):
	variants = []
	for filename in os.listdir(puzzlesDirectory):
		for strategy in strategies:
			if strategy == "astr":
				variants = astrVariants
			else:
				variants = lrudVariants
			for variant in variants:
				os.system("python main.py {} {} {} {} {}".format
				(
					strategy,
					variant,
					os.path.join(puzzlesDirectory, filename),
					os.path.join( solutionsDirectory, packFilename(filename, strategy, variant, "sol" ) ),
					os.path.join( statsDirectory, packFilename(filename, strategy, variant, "stats") )
				))

def dictUpdate(dict, argument):
	dict.update(argument)
	return dict

def listAppend(list, argument):
	list.append(argument)
	return list

def combineGlobalMeansStats(bfsStats, dfsStats, astrStats, iterations):
	globalStats = dictUpdate({}, {stat:dictUpdate({}, {strategy:dictUpdate({}, {it:0 for it in range(1, iterations + 1)}) for strategy in strategies} ) for stat in stats})
	for stat in globalStats:
		for strategy in stat:
			for key, iter in strategy:
				if strategy.key() == "bfs":
					strategy[key] = mean([bfsStats[stat.key()][variant][key] for variant in lrudVariants])
				elif strategy.key() == "dfs":
					strategy[key] = mean([dfsStats[stat.key()][variant][key] for variant in lrudVariants])
				elif strategy.key() == "astr":
					strategy[key] = mean([astrStats[stat.key()][variant][key] for variant in astrVariants])
	return globalStats

def loadStats(statsDirectory, iterations):
	bfsStats = dictUpdate({}, {stat:dictUpdate({}, {variant:dictUpdate({}, {it:[] for it in range(1, iterations + 1)}) for variant in lrudVariants} ) for stat in stats})
	dfsStats = bfsStats.copy()
	astrStats = dictUpdate({}, {stat:dictUpdate({}, {variant:dictUpdate({}, {it:[] for it in range(1, iterations + 1)}) for variant in astrVariants} ) for stat in stats})
	for filename in os.listdir(statsDirectory):
		iterations, strategy, variant, type = unpackFilename(filename)
		iterations = int(iterations)
		with open(os.path.join(statsDirectory, filename), 'r') as f:
			for stat in stats:
				value = f.readline()
				if stat != "time":
					value = int(value)
				else:
					value = float(value)
				if strategy == "bfs":
					bfsStats[stat][variant][iterations].append(value)
				elif strategy == "dfs":
					dfsStats[stat][variant][iterations].append(value)
				else:
					astrStats[stat][variant][iterations].append(value)
	return bfsStats, dfsStats, astrStats

def calculateMeans(stats):
	for stat in stats.values():
		for variant in stat.values():
			for key, iter in variant.items():
				variant[key] = mean(iter)


#solvePuzzles("wikamp/puzzles", "wikamp/solutions", "wikamp/stats")
bfsStats, dfsStats, astrStats = loadStats("wikamp/stats", 3)

print(bfsStats)
print(dfsStats)
print(astrStats)

calculateMeans(bfsStats)
calculateMeans(dfsStats)
calculateMeans(astrStats)
#print(combineGlobalStats(bfsStats, dfsStats, astrStats, 3))


{'length': {'RDUL': {1: [1, 1, 1, 1], 2: [2, 2, 2, 2, 2, 10], 3: []}, 'RDLU': {1: [1, 1, 1, 1], 2: [2, 2, 2, 2, 2, 10], 3: []}, 'DRUL': {1: [1, 1, 1, 1], 2: [2, 2, 2, 10, 2, 2], 3: []}, 'DRLU': {1: [1, 1, 1, 1], 2: [2, 2, 2, 10, 2, 2], 3: []}, 'LUDR': {1: [1, 11, 1, 11], 2: [2, 12, 2, 2, 2, 2], 3: []}, 'LURD': {1: [1, 11, 1, 11], 2: [2, 12, 2, 2, 2, 10], 3: []}, 'ULDR': {1: [1, 11, 1, 11], 2: [2, 2, 2, 10, 2, 2], 3: []}, 'ULRD': {1: [1, 11, 1, 11], 2: [2, 2, 2, 2, 2, 2], 3: []}}, 'visitedStates': {'RDUL': {1: [4, 4, 4, 4], 2: [16, 8, 20, 8, 28, 40], 3: []}, 'RDLU': {1: [4, 4, 4, 4], 2: [16, 8, 20, 8, 28, 7860], 3: []}, 'DRUL': {1: [4, 4, 4, 4], 2: [16, 8, 28, 7860, 20, 8], 3: []}, 'DRLU': {1: [4, 4, 4, 4], 2: [16, 8, 28, 40, 20, 8], 3: []}, 'LUDR': {1: [12, 63232, 12, 106448], 2: [36, 58116, 56, 113172, 48, 77108], 3: []}, 'LURD': {1: [12, 80284, 12, 102476], 2: [36, 64908, 48, 68936, 56, 92548], 3: []}, 'ULDR': {1: [12, 102476, 12, 80284], 2: [36, 90824, 56, 92548, 48, 68936], 3: []},

TypeError: object of type 'float' has no len()